# LAB5: YOLO implementation
## Based on the given code, write a Python program using the OpenCV, TensorFlow and Keras libraries that performs the following functions:
1. Implement the YOLO algorithm in the version of your choise.
2. On the photo indicated in the commandline, the algorithm will indicate the detected objects and mark the on the screen.
3. On the film indicated in the commandline it will indicate the detected objects and mark the on the screen.
4. The object should be marked as an appropriate envelope and class symbol throughout all the duration of the movie.
### Please send the program code and the results of the code (photo and video with tagged objects) via Teams at the time specified in the assignment.
### Good luck
#### Mateusz Andrzejewski 